In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import keras
from keras import backend as K
from keras.layers import Input, Dense, Dropout
from keras.layers import PReLU
from keras.models import Model
from tensorflow.keras.optimizers import Adamax
import math
import matplotlib
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import matplotlib
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [ ]:

# Model fixed parameters
NInputs = 4 # Number of features (model inputs)
NOutputs = 3 # Number of targets (model outputs)



In [ ]:

# Import training data (model inputs & outputs)
Feature_Train = pd.read_csv('processed_data/SL_Train_Features.csv', header = 0)
Feature_Train = Feature_Train[['GR', 'RHOB','NPHI','RPCELM_l10']]

Feature_Train = np.asarray(Feature_Train)
print(Feature_Train.shape)
Target_Train = pd.read_csv('processed_data/SL_Train_Target.csv', header = 0)
Target_Train = Target_Train[['SW', 'PHIF', 'VSH']]
Target_Train = np.asarray(Target_Train)
print(Target_Train.shape)
print(Feature_Train[0])


# Import validation data (model inputs & outputs)
Feature_Val = pd.read_csv('processed_data/SL_Val_Features.csv', header = 0)
Feature_Val = Feature_Val[['GR', 'RHOB','NPHI','RPCELM_l10']]

Feature_Val = np.asarray(Feature_Val)
print(Feature_Val.shape)
Target_Val = pd.read_csv('processed_data/SL_Val_Target.csv', header = 0)
Target_Val = Target_Val[['SW', 'PHIF', 'VSH']]
Target_Val = np.asarray(Target_Val)
print(Target_Val.shape)



# Import test data (model inputs & outputs)
Feature_Test = pd.read_csv('processed_data/SL_Test_Features.csv', header = 0)
Feature_Test = Feature_Test[['GR', 'RHOB','NPHI','RPCELM_l10']]

Feature_Test = np.asarray(Feature_Test)
print(Feature_Test.shape)
Target_Test = pd.read_csv('processed_data/SL_Test_Target.csv', header = 0)
Target_Test = Target_Test[['SW', 'PHIF', 'VSH']]
Target_Test = np.asarray(Target_Test)
print(Target_Test.shape)



In [ ]:
Feature_Test

In [ ]:
# Defining neural network

rate = 0.1

def nn():
    NN_ip = Input(shape=(NInputs,))
    x = Dense(1000)(NN_ip)
    x = PReLU()(x)
    x = Dropout(rate)(x)
    x = Dense(1000)(x)
    x = PReLU()(x)
    x = Dropout(rate)(x)
    NN_op = Dense(NOutputs)(x)
       
    NN_model = Model(NN_ip, NN_op)
    
    def loss_mse(true, pred):
        mean = pred[:, :NOutputs]
        return K.mean(K.square(true - mean), -1)

    def metric_mse(y_true, y_pred):
        mean = y_pred[:, :NOutputs]
        return K.mean(K.square(y_true - mean), -1)
    
    opt = Adamax(1e-3)
    
    NN_model.compile(loss = loss_mse, optimizer = opt, metrics = [metric_mse])
    
    return NN_model

NN = nn()
NN.summary()


In [ ]:
NN = nn()
# NN.load_weights('NNWeights.h5')

history = NN.fit(Feature_Train, Target_Train, 
                     epochs = 1000, 
                     batch_size = int(Feature_Train.shape[0]/8), 
                     shuffle = True, 
                     validation_data = (Feature_Val, Target_Val))
    
plt.plot(history.history['val_loss'][2:],'r')
plt.plot(history.history['loss'][2:],'k')


In [ ]:
#3.1 Plot the predicted saturation versus the Interpreted saturation
PredictedTarget = NN.predict(Feature_Test)
[PredictedSw, PredictedPHIF, PredictedVSH] = [PredictedTarget[:,0], PredictedTarget[:,1], PredictedTarget[:,2]]


In [ ]:
PredictedTarget

In [ ]:
PredictedTarget.shape
Target_Test.shape

In [ ]:
plt.figure(figsize = (6,6))
plt.style.use('seaborn-white')
plt.scatter(Target_Test[:,0], PredictedSw)
plt.plot(np.linspace(0, 1, 2), np.linspace(0, 1, 2), color = 'r', alpha = 1)
plt.xlabel('Interpreted Sw', FontSize = 16)
plt.ylabel('Predicted Sw', FontSize = 16)
plt.show()


In [ ]:
plt.figure(figsize = (6,6))
plt.style.use('seaborn-white')
plt.scatter(Target_Test[:,1], PredictedPHIF)
plt.plot(np.linspace(0, .4, 2), np.linspace(0, .4, 2), color = 'r', alpha = 1)
plt.xlabel('Interpreted PHIF', FontSize = 16)
plt.ylabel('Predicted PHIF', FontSize = 16)
plt.show()


In [ ]:
plt.figure(figsize = (6,6))
plt.style.use('seaborn-white')
plt.scatter(Target_Test[:,2], PredictedVSH)
plt.plot(np.linspace(0, .8, 2), np.linspace(0, .8, 2), color = 'r', alpha = 1)
plt.xlabel('Interpreted VSH', FontSize = 16)
plt.ylabel('Predicted VSH', FontSize = 16)
plt.show()

In [ ]:
output_df = pd.read_csv('processed_data/SL_Test_Target.csv', header = 0)
type(output_df)
output_df.insert( 4,"PredictedSw", PredictedSw)
output_df.insert( 6,"PredictedVSH", PredictedVSH)
output_df.insert( 8,"PredictedPHIF", PredictedPHIF)
output_df
output_df.to_csv('processed_data/SL_PredictedProperties.csv', index=True)


In [ ]:
output_df['VSHDifference'] = abs(output_df['PredictedVSH'] - output_df['VSH'])
output_df['PHIFDifference'] = abs(output_df['PredictedPHIF'] - output_df['PHIF'])
output_df['SwDifference'] = abs(output_df['PredictedSw'] - output_df['SW'])

## Plotting the Prediction Results
After the prediction has completed, we can view the predicted result against the actual measurement on the test well: 15/9-F-11 B

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20,10))

ax1 = plt.subplot2grid((2,1), (0,0))
ax2 = plt.subplot2grid((2,1), (1,0))

ax1.plot(output_df['MD'], output_df['VSH'], label='Interpreted', color='black')
ax1.plot(output_df['MD'], output_df['PredictedVSH'], label='Predicted', color='red')

ax1.set_ylabel('VSH', fontsize=16, fontweight='bold', labelpad=30)
ax1.set_xlabel('Depth (m)', fontsize=16, fontweight='bold', labelpad=30)
ax1.tick_params(which='both', labelsize=14)


ax2.plot(output_df['MD'], output_df['VSHDifference'], label='Interpreted', color='black')
ax2.set_ylim(0,1)

ax2.set_ylabel('Abs. Prediction Difference', fontsize=16, fontweight='bold', labelpad=30)

for ax in [ax1, ax2]:
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_xlabel('Depth (m)', fontsize=16, fontweight='bold', labelpad=30)
    ax.tick_params(which='both', labelsize=14)

ax1.legend(fontsize=14, facecolor='white', frameon=True)

plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20,10))

ax1 = plt.subplot2grid((2,1), (0,0))
ax2 = plt.subplot2grid((2,1), (1,0))

ax1.plot(output_df['MD'], output_df['PHIF'], label='Interpreted', color='black')
ax1.plot(output_df['MD'], output_df['PredictedPHIF'], label='Predicted', color='red')

ax1.set_ylabel('PHIF', fontsize=16, fontweight='bold', labelpad=30)
ax1.set_xlabel('Depth (m)', fontsize=16, fontweight='bold', labelpad=30)
ax1.tick_params(which='both', labelsize=14)


ax2.plot(output_df['MD'], output_df['PHIFDifference'], label='Interpreted', color='black')
ax2.set_ylim(0,1)

ax2.set_ylabel('Abs. Prediction Difference', fontsize=16, fontweight='bold', labelpad=30)

for ax in [ax1, ax2]:
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_xlabel('Depth (m)', fontsize=16, fontweight='bold', labelpad=30)
    ax.tick_params(which='both', labelsize=14)

ax1.legend(fontsize=14, facecolor='white', frameon=True)

plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(20,10))

ax1 = plt.subplot2grid((2,1), (0,0))
ax2 = plt.subplot2grid((2,1), (1,0))

ax1.plot(output_df['MD'], output_df['SW'], label='Interpreted', color='black')
ax1.plot(output_df['MD'], output_df['PredictedSw'], label='Predicted', color='red')

ax1.set_ylabel('Water Saturation (dec)', fontsize=16, fontweight='bold', labelpad=30)
ax1.set_xlabel('Depth (m)', fontsize=16, fontweight='bold', labelpad=30)
ax1.tick_params(which='both', labelsize=14)
ax1.set_xlim(4000,4500)


ax2.plot(output_df['MD'], output_df['SwDifference'], label='Interpreted', color='black')
ax2.set_ylim(0,1)

ax2.set_ylabel('Abs. Prediction Difference', fontsize=16, fontweight='bold', labelpad=30)

for ax in [ax1, ax2]:
    ax.grid()
    ax.set_axisbelow(True)
    ax.set_xlabel('Depth (m)', fontsize=16, fontweight='bold', labelpad=30)
    ax.tick_params(which='both', labelsize=14)

ax1.legend(fontsize=14, facecolor='white', frameon=True)
